# Arthas简介

[Arthas](https://arthas.gitee.io/index.html)（阿尔萨斯） 能为你做什么？

![](assets/arthas.png)

Arthas 是Alibaba开源的Java诊断工具，深受开发者喜爱。

当你遇到以下类似问题而束手无策时，Arthas可以帮助你解决：

1. 这个类从哪个 jar 包加载的？为什么会报各种类相关的 Exception？`sc`
2. 我改的代码为什么没有执行到？难道是我没 commit？分支搞错了？ `trace/jad`
3. 遇到问题无法在线上 debug，难道只能通过加日志再重新发布吗？  `redefine`
4. 线上遇到某个用户的数据处理有问题，但线上同样无法 debug，线下无法重现！ `tt`
5. 是否有一个全局视角来查看系统的运行状况？ `dashboard`
6. 有什么办法可以监控到JVM的实时运行状态？  `jvm/dashboard/thread`
7. 怎么快速定位应用的热点，生成火焰图？ `profiler`

Arthas支持JDK 6+，支持Linux/Mac/Winodws，采用命令行交互模式，同时提供丰富的 Tab 自动补全功能，进一步方便进行问题的定位和诊断。

In [1]:
import socket
import requests

def run(command):
    client = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 
    client.connect(('localhost',8999))
    client.send(command.encode('utf8'))
    client.close()

def trigger(name):
    url = f'http://127.0.0.1:8080/{name}'
    print(requests.get(url).text)
    

# 原理简介

## Java Agent
Java Agent 是在 JDK1.5 引入的，是一种可以动态修改 Java 字节码的技术。
Java 类编译之后形成字节码被 JVM 执行，在 JVM 在执行这些字节码之前获取这些字节码信息，并且通过字节码转换器对这些字节码进行修改，来完成一些额外的功能。

## 功能
* Java Agent 能够在加载 Java 字节码之前进行拦截并对字节码进行修改;
* 在 Jvm 运行期间修改已经加载的字节码;

常用于：
* class文件加密
* 性能监控

# Java Agent

## 相关的开源工具和项目
* arthas, 在线排查问题，动态追踪Java代码，实时监控JVM
* Skywalking，Apache的链路追踪工具
* jvm-profiler，Uber的JVM采集工具，收集各项指标

## 原理
字节码转换器的执行方式有两种：
1. `main`执行之前，`premain`的方式
2. 程序执行中，`attach`的方式

## 相关概念
* JVMTI  
> JVM Tool Interface, 是JVM暴露给用户的回调借口，基于事件驱动，是Debugger、Profiler、Monitor、Thread Analyser 等工具的基础
* JVMTIAgent
> 一般通过Agent的方式使用JVMTI，设置一些回调函数从而获得JVM的相关信息
* Instrument Agent
> JVMTI Agent的一个实现，支持启动、运行期加载agent
* JVM Attach
> JVM 提供的一种 JVM 进程间通信的功能，能让一个进程传命令给另一个进程，并进行一些内部的操作。比如dump、jstack

## 实现Java Agent的框架
* ASM，直接生产class字节码文件
* Javassist，使用Java编码的形式操作字节码
* Instrument，JVM提供的支持修改已加载类的工具
* Byte Buddy，提供了类型安全的API和注解，简化复杂的字节码操作

[Arthas原理](https://www.jianshu.com/p/70c1c55f12ef)

# 安装

```shell
curl -sk https://arthas.gitee.io/arthas-boot.jar -o ~/.arthas-boot.jar \
&& echo "alias as.sh='java -jar ~/.arthas-boot.jar --repo-mirror aliyun --use-http'" >> ~/.bashrc \
&& source ~/.bashrc
```

> -s: 不输出错误和进度 -k: 跳过SSL检测 -o: 存储为文件

## 启动
```
as.sh [pid]
或
java -jar ~/.arthas-boot.jar [pid]
```


# 命令列表
[官网命令列表](https://arthas.gitee.io/commands.html)

## jvm相关
* `dashboard`——当前系统的实时数据面板
* `thread`——查看当前 JVM 的线程堆栈信息
* `jvm`——查看当前 JVM 的信息
* `sysprop`——查看和修改JVM的系统属性
* `sysenv`——查看JVM的环境变量
* `vmoption`——查看和修改JVM里诊断相关的option
* `perfcounter`——查看当前 JVM 的Perf Counter信息
* `logger`——查看和修改logger
* `getstatic`——查看类的静态属性
* `ognl`——执行ognl表达式
* `mbean`——查看 Mbean 的信息
* `heapdump`——dump java heap, 类似jmap命令的heap dump功能
## class/classloader相关
* `sc`——查看JVM已加载的类信息
* `sm`——查看已加载类的方法信息
* `jad`——反编译指定已加载类的源码
* `mc`——内存编译器，内存编译.java文件为.class文件
* `redefine`——加载外部的.class文件，redefine到JVM里
* `dump`——dump 已加载类的 byte code 到特定目录
* `classloader`——查看classloader的继承树，urls，类加载信息，使用classloader去getResource
## monitor/watch/trace相关
请注意，这些命令，都通过字节码增强技术来实现的，会在指定类的方法中插入一些切面来实现数据统计和观测，因此在线上、预发使用时，请尽量明确需要观测的类、方法以及条件，诊断结束要执行 stop 或将增强过的类执行 reset 命令。
* `monitor`——方法执行监控
* `watch`——方法执行数据观测
* `trace`——方法内部调用路径，并输出方法路径上的每个节点上耗时
* `stack`——输出当前方法被调用的调用路径
* `tt`——方法执行数据的时空隧道，记录下指定方法每次调用的入参和返回信息，并能对这些不同的时间下调用进行观测
## profiler/火焰图
* `profiler`–使用async-profiler对应用采样，生成火焰图

# 示例

## 死锁排查

使用 `thread`命令排查死锁问题(阻塞问题)

> 死锁的四个必要条件:互斥条件、请求和保持条件、不可抢占条件、循环等待条件

```java
public void doDeadLock() {
    new Thread(() -> {
        resourceA.lock();
        sleep(1000);
        resourceB.lock();
    }).start();
    new Thread(() -> {
        resourceB.lock();
        sleep(1000);
        resourceA.lock();
    }).start();
}
```

* 传统的方式: `jstack`  
`jstack [pid]`  
信息非常多且杂乱

In [2]:
# 1. 触发死锁程序
trigger('dead-lock')

OK


In [8]:
# 2. 查看阻塞线程
run('thread -b')
# 查看线程堆栈信息
# run('thread [tid]') 

In [5]:
# 3. 释放进程资源
trigger('release-dead-lock')

OK


## 更新日志级别
使用`logger`命令查看日志信息，更新日志级别

Usage:
```
logger [-c classloader] [-l level] [-n name]
```

* 传统方式
1. 改配置，重启
2. 使用logback等，支持配置热加载的日志组件
3. 预留后门

In [12]:
# 查看日志相关配置信息
run('logger')

In [13]:
# 记录日志
trigger('start-logger')

OK


In [ ]:
# 设置日志级别
# run('logger -l info -n *** -c ***')

In [14]:
# 关闭日志记录
trigger('stop-logger')

OK


## OGNL 为所欲为
### 什么是OGNL
OGNL 是 Object-Graph Navigation Language 的缩写（对象导航图语言），从语言角度来说：它是一个功能强大的表达式语言（EL），用来获取和设置 java 对象的属性 , **它旨在提供一个更高抽象度语法来对 java 对象图进行导航**，OGNL 在许多的地方都有应用

[官网](https://commons.apache.org/proper/commons-ognl/language-guide.html)
### OGNL表达式
1. 常量 字符串:"ello"、字符:'h'、数字:int,long,float,double,BigInteger,BigDecimal、布尔值:ture|false、null
2. 属性的引用 例如：user.name
3. 变量的引用 例如：#name
4. 静态变量的访问 使用 @class@field
5. 静态方法的调用 使用 @class@method(args), 如果没有指定 class 那么默认就使用 java.lang.Math.
6. 构造函数的调用 例如：new java.util.ArrayList();

### watch 观察方法调用
USAGE:
```
watch [-befs]  [-n <times>] class-pattern method-pattern [express] [condition-express]

watch org.apache.commons.lang.StringUtils isBlank '{params, target, returnObj}' -x 2
```
观察到指定方法的调用情况。能观察到的范围为：返回值、抛出异常、入参，通过编写 `OGNL` 表达式进行对应变量的查看。

匹配表达式、观察表达式都是围绕着一个 Arthas 中的通用通知对象[表达式核心变量](https://arthas.gitee.io/advice-class.html)
```java
public class Advice {
    private final ClassLoader loader;
    private final Class<?> clazz;
    private final ArthasMethod method;
    private final Object target;
    private final Object[] params;
    private final Object returnObj;
    private final Throwable throwExp;
    private final boolean isBefore;
    private final boolean isThrow;
    private final boolean isReturn;
    // getter/setter  
}  
```

In [51]:
# 创建用户
trigger('create-user')

{"name":"黄天翊","birthday":"1999-08-09","male":true,"age":21}


In [33]:
# 观察创建用户
# run("watch *WatchDemo supplyUser '{returnObj}' -n 2 -x 1 '1==1'")
# run("watch *WatchDemo supplyUser '{returnObj}' -n 2 -x 1 'returnObj.age>18'")
# run("watch *WatchDemo supplyUser '{class,method,target,params,returnObj}' -x2 -n2")
run("watch *WatchDemo supplyUser 'target.userCount' -n 5")

---

In [43]:
# 消费成年人
trigger('consume-adult')

禁止消费未成年人


In [41]:
# 观察消费用户
# run("watch *WatchDemo consumeUser '{returnObj, params[0].age, throwExp}' -n 5 -x 2")
run("watch *WatchDemo consumeUser '{params[0].name, params[0].age, throwExp}' 'params[0].age<18' -n 1")
# run("watch *WatchDemo consumeUser '{params[0].name, params[0].age, throwExp}' -e")

---

In [47]:
# 触发复杂方法
trigger('complex-user')

{"19":[{"name":"龙绍齐","birthday":"2001-05-16","male":true,"age":19}],"20":[{"name":"田明轩","birthday":"2000-05-26","male":false,"age":20},{"name":"于晓啸","birthday":"2000-04-23","male":true,"age":20}],"21":[{"name":"沈锦程","birthday":"1999-12-19","male":true,"age":21},{"name":"洪昊天","birthday":"1999-08-22","male":false,"age":21}],"23":[{"name":"唐文昊","birthday":"1997-03-06","male":true,"age":23},{"name":"石金鑫","birthday":"1997-02-08","male":false,"age":23}],"24":[{"name":"韩天宇","birthday":"1996-04-06","male":true,"age":24},{"name":"贾思源","birthday":"1996-05-10","male":true,"age":24}]}


In [44]:
# run("watch *WatchDemo adultUserGroup 'returnObj' -x 2")

# 观察20岁的男性用户
cmd = """
watch *WatchDemo adultUserGroup
'{returnObj.size, returnObj.keys,
returnObj[20]==null?{}:returnObj[20].{? #this.male},
params[0].{? #this.male && #this.age==20}.{name}}'
-x2
"""
run(cmd.replace('\n',' '))

In [48]:
# 高级用法
cmd = """
watch *WatchDemo supplyUser
'#springContext=@top.abosen.toys.arthasdemo.context.ApplicationContextHelper@applicationContext,
{#springContext.getBean("watchDemo").consumeUser(returnObj), target.userCount, returnObj}'
'returnObj.age>18' -n3 -x2
"""
run(cmd.replace('\n',' '))

### Monitor 执行方法监控
`monitor` 对匹配 class-pattern／method-pattern的类、方法的调用进行监控  
Usage:
```
monitor [-c <interval>] [-n <times>] [-E <value>] class-pattern method-pattern
monitor org.apache.commons.lang.StringUtils isBlank -c 5
```

In [52]:
# 监测这个可能异常的方法
run('monitor -c1 -n10 *WatchDemo consumeUser ')

In [53]:
from time import sleep
for i in range(0,100):
    trigger('consume-adult')
    sleep(0.1)

禁止消费未成年人
禁止消费未成年人
薛明辉
雷熠彤
王旭尧
傅哲瀚
方果
邱鸿煊
严锦程
覃乐驹
卢健柏
禁止消费未成年人
蔡笑愚
曾烨华
尹鹏
禁止消费未成年人
吕伟诚
禁止消费未成年人
吴志泽
禁止消费未成年人
谢远航
熊鸿涛
龚嘉懿
方煜城
方修洁
孙峻熙
禁止消费未成年人
程睿渊
杜文轩
禁止消费未成年人
蒋风华
禁止消费未成年人
禁止消费未成年人
禁止消费未成年人
邵修洁
禁止消费未成年人
禁止消费未成年人
禁止消费未成年人
禁止消费未成年人
覃立轩
顾风华
郝明哲
丁志泽
曹弘文
许伟泽
吴伟泽
禁止消费未成年人
钱伟祺
沈思聪
林航
禁止消费未成年人
禁止消费未成年人
田钰轩
唐鹏煊
钟潇然
禁止消费未成年人
吴健雄
郭越彬
钟伟宸
禁止消费未成年人
禁止消费未成年人
史明杰
禁止消费未成年人
禁止消费未成年人
禁止消费未成年人
禁止消费未成年人
夏嘉懿
禁止消费未成年人
杜昊天
禁止消费未成年人
禁止消费未成年人
禁止消费未成年人
冯瑞霖
禁止消费未成年人
禁止消费未成年人
邓荣轩
禁止消费未成年人
禁止消费未成年人
罗荣轩
谭炎彬
禁止消费未成年人
何烨磊
禁止消费未成年人
禁止消费未成年人
叶晓博
禁止消费未成年人
禁止消费未成年人
傅哲瀚
于越泽
禁止消费未成年人
尹越彬
宋思聪
吴健柏
侯修洁
禁止消费未成年人
徐远航
禁止消费未成年人
于文
禁止消费未成年人
禁止消费未成年人


### tt 时空隧道，回溯调用
`tt`开启 方法执行数据的时空隧道，记录下指定方法每次调用的入参和返回信息，
并能对这些不同的时间下调用进行观测，复现场景。  
[USAGE](https://arthas.gitee.io/tt.html):
```
tt [-tpi] [class-pattern] [method-pattern] [condition-express]
```
tt的问题:
1. ThreadLocal 信息丢失
    很多框架偷偷的将一些环境变量信息塞到了发起调用线程的 ThreadLocal 中，由于调用线程发生了变化，这些 ThreadLocal 线程信息无法通过 Arthas 保存，所以这些信息将会丢失。

2. 引用的对象
    需要强调的是，tt 命令是将当前环境的对象引用保存起来，但仅仅也只能保存一个引用而已。如果方法内部对入参进行了变更，或者返回的对象经过了后续的处理，那么在 tt 查看的时候将无法看到当时最准确的值。这也是为什么 watch 命令存在的意义。

In [56]:
# 记录三次调用
# run('tt -t *WatchDemo consumeUser -n10')

# 观察调用信息
# run('tt -i 1000')
# 重放调用,适合微服务调试
# run('tt -i 1000 -p')

# 记录三次男性异常调用
# run('tt -t -n3 *WatchDemo consumeUser "isThrow && params[0].male"')
# 观察这些调用参数
run("tt -s 'isThrow && params[0].male' -w params[0]")

In [55]:
from time import sleep
for i in range(0,10):
    trigger('consume-adult')
    sleep(0.1)

冯靖琪
邓文轩
苏睿渊
禁止消费未成年人
禁止消费未成年人
杨文博
禁止消费未成年人
禁止消费未成年人
高明辉
禁止消费未成年人


### stack 调用路径
输出当前方法被调用的调用路径  
USAGE:
```
stack [-n <times>] [-E] class-pattern [method-pattern] [condition-express]
stack *StringUtils isBlank params[0].length==1
```

In [50]:
# 查看正常返回方法的调用路径
# run('stack *WatchDemo consumeUser isReturn')
trigger('consume-adult')

谢天宇


### trace 链路追踪
USAGE:
    
```
trace [-n <times>] [-p <path>] [-E] [--skipJDKMethod <true|false>] class-pattern method-pattern [condition-express]

trace *StringUtils isBlank '#cost>100'
```
trace 命令能主动搜索 class-pattern／method-pattern 对应的方法调用路径，渲染和统计整个调用链路上的所有性能开销和追踪调用链路。


In [90]:
trigger('analyze-user')

{"total":100,"male":45,"female":55,"mostMaleBirthMonth":5,"mostFemaleBirthMonth":10}


In [89]:
# run("trace *TraceDemo analyze -n1 ")
# run("trace *TraceDemo analyze -n1 --skipJDKMethod false")
# 正则匹配，一次性匹配多个方法
run("trace -E .*TraceDemo|.*UserRepository analyze|getOne -n1 --skipJDKMethod false")

## 热更新
### jad,mc,redefine
USAGE:  
```
jad [-c <classloader hash>] [--hideUnicode] [-E] [--source-only] class-pattern [method-name]
jad --source-only java.lang.String
```
jad返回带有location,classloader信息,通常使用`jad --source-only class-pattern > /tmp/TheClass.java` 再配合`mc/redefine`使用


In [ ]:
# 反编译成java文件，并修改
run('jad --source-only *TraceDemo > /tmp/TraceDemo.java')

In [ ]:
# 内存编译
run('mc /tmp/TraceDemo.java -d /tmp')

In [ ]:
# 重新加载class文件
run('redefine /tmp/top/abosen/toys/arthasdemo/ognl/TraceDemo.class')
# 或者从ide进行编译，可以链接其他类，供编译器进行相关类型推导
# run('redefine /Users/qiubaisen/Documents/gitproject/arthas-demo/build/classes/java/main/top/abosen/toys/arthasdemo/ognl/TraceDemo.class')

In [65]:
# 重新注册trigger, invokedynamic动态生产的类已经被替换掉了
cmd = """
ognl '#springContext=@top.abosen.toys.arthasdemo.context.ApplicationContextHelper@applicationContext,
#springContext.getBean("traceDemo").init()'
"""
run(cmd.replace('\n',' '))

In [87]:
trigger('analyze-user')

{"total":100,"male":45,"female":55,"mostMaleBirthMonth":7,"mostFemaleBirthMonth":12}


**`redefine`的坑**
1. 自身局限
    * 不允许新增加field/method
    * 正在跑的函数，没有退出不能生效
2. `redefine`与`watch,tt,trace,jad,monitor`等增强可能冲突  
    * 执行完redefine之后，如果再执行上面提到的命令，则会把redefine的字节码重置。 
    * reset 命令对 redefine 无效。
    * 参考[jad原理](https://github.com/alibaba/arthas/issues/763)

3. [lambda导致redefine失败](https://www.robberphex.com/lambda-causes-arthas-cant-redefine/)

    `lambda`编译后的方法签名`lambda$<methodname>$<lambdaCount>`,`jdk8u74-b02`版本后,`lambdaCount`按类单独计数，不再全局统一。
    * 删除、新增lambda会创建新的方法，当前的`redefine`不支持。 删除lambda可以通过增加`if`条件，越过lambda而非删除完成。
    * 线上编译环境与本地版本不一致，可以本地编译后，上传`class`文件，再使用`redefine`, 不可上传文件的环境参考[BASE64上传class文件](https://github.com/alibaba/arthas/issues/372)
    * `lambda`方法返回值类型在`redefine`过程中也算作签名，参考[arthas redefine的坑](https://www.yuque.com/abosen/blogs/guuchv)

## profiler
使用`async-profiler`生成火焰图。
`profiler`命令支持生成应用热点的火焰图。本质上是通过不断的采样，然后把收集到的采样结果生成火焰图。
 
USAGE:
```
profiler [--allkernel] [--alluser] [-d <time>] [-e <event>] [-f <file name>] [--format <file format>] [-h] [-i <interval>] [--threads] action [actionArg]
```

[使用Chrome观察结果](http://localhost:3658/arthas-output/)


In [92]:
# 1.运行要监测的热点程序
trigger('start-profiler')

OK


In [94]:
# 2.使用profiler命令,对内存分配采样10s
run('profiler -d 10 -e alloc -f arthas-output/output-alloc.svg start')

In [93]:
# 3.对cpu指标进行采样
run('profiler -d 10 -e cpu -f arthas-output/output-cpu.svg  start')

In [95]:
# 4.停止热点程序
trigger('stop-profiler')

OK


## 后台执行
1. `&` 后台执行
2. `jobs` 查看后台任务
3. `ctrl z`暂停任务(当前版本与zsh冲突), `ctrl c`取消任务
4. `fg,bg` 把任务置于前后台执行
5. `kill` 关闭任务
6. `quit` 退出arthas，不关闭后台


In [ ]:
run('watch *WatchDemo consumeUser {params[0]} isThrow >> /tmp/async.out &')

# 参考
1. [Arthas官网命令详解](https://arthas.gitee.io/commands.html)
2. [Arthas IDEA 插件](https://plugins.jetbrains.com/plugin/13581-arthas-idea)
3. [Alibaba Cloud Toolkit](https://plugins.jetbrains.com/plugin/11386-alibaba-cloud-toolkit)
4. [Async-profiler 实践](https://www.jianshu.com/p/918e1dce61cd)
5. [Apache OGNL](https://commons.apache.org/proper/commons-ognl/language-guide.html)
6. [jad原理](https://github.com/alibaba/arthas/issues/763)
7. [BASE64上传class文件](https://github.com/alibaba/arthas/issues/372)
8. [lambda导致redefine失败](https://www.robberphex.com/lambda-causes-arthas-cant-redefine/)
9. [表达式核心变量](https://arthas.gitee.io/advice-class.html)
10. [Artahs redefine踩坑](https://www.yuque.com/abosen/blogs/guuchv)
11. [Spring Mvc Restul调优](https://www.yuque.com/abosen/blogs/bfzss2)